In [ ]:
## Load the environment variables
import langchain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
groq_api_key = os.getenv('GROQ_API_KEY')
## Langchain tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY") 
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [7]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [ ]:
# Load LLM model
from langchain_groq import ChatGroq
model = ChatGroq(model="Llama3-8b-8192",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B4179FAC50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B4179FBD90>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# Import libraries
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



c:\Projects\Chatbots with message history\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
## Define huggingface embedding technique
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
#Load the Data from Web

import bs4
web = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    )
)
docs = web.load()

In [ ]:
## Split the documents into small chunks

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splited_docs = splitter.split_documents(docs)

In [ ]:
## Store the documnets in vector DB

db = Chroma.from_documents(splited_docs,embedding=embeddings)

In [ ]:
## Convert VectorDB to runnable retrivers
retrivers = db.as_retriever()
retrivers

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001B472563640>, search_kwargs={})

In [ ]:
## Define Chat Prompt template

system_prompt = (
    "You are a helpfull assistant for question-answering tasks"
    "Use the following prieces of retrived contex to answer"
    "the question. If you dont know the answer, sat that you dont know"
    "Use three sentences maximum and keep the answer concize"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [ ]:
##Create the RAG Chain
qachain = create_stuff_documents_chain(model,prompt)
rag_chain = create_retrieval_chain(retrivers,qachain)

In [ ]:
## Test the RAG Chain
responce = rag_chain.invoke(
    {"input":"what is Self-Reflection"}
)
responce

{'input': 'what is Self-Reflection',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Another quite distinct approach, LLM

In [51]:
responce["answer"]

'Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [ ]:
#Adding Chat history & and creating history aware retrivers

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
contextulaized_qa_system_prompt = ( 
    "Given the chat history and latest user question"
    "which might might reference context in chat history"
    "formulate a standalone question that can be understood"
    "without chat history. Do NOT answer the question"
    "just reformulate if needed and otherwise return it as it is"
)
contextualize_qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextulaized_qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)
history_aware_retriver = create_history_aware_retriever(model,retrivers,contextualize_qa_prompt)
history_aware_retriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001B472563640>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [ ]:
#Define Q&A Promptwith cht history
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [ ]:
qa_chain = create_stuff_documents_chain(model, )
rag_chain = create_retrieval_chain(history_aware_retriver,qa_chain)

In [ ]:
## Test the History Aware Q&A ChatBot
from langchain_core.messages import HumanMessage,AIMessage
chat_history = []
question1 = "what is Self-Reflection"
responce1 = rag_chain.invoke({"input":question1,"chat_history":chat_history})
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=responce1["answer"])
    ]
)
question2 = "tell me more about it"
responce2 = rag_chain.invoke({"input":question2,"chat_history":chat_history})
responce2["answer"]

"Self-Reflection is a process in the Reflexion framework that enables agents to reflect on their past actions and decisions. It does this by presenting the agent with a pair of examples, one of which is a failed trajectory and the other is an ideal reflection for guiding future changes in the plan. These reflections are then added to the agent's working memory, up to three, to be used as context for querying the Language Model (LM)."

In [63]:
chat_history

[HumanMessage(content='what is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.', additional_kwargs={}, response_metadata={})]

In [ ]:
## Manage History and User Sessions 

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "xyz123"}
    },  # constructs a key "xyz123" in `store`.
)["answer"]

'Task decomposition is a process in which an agent breaks down a complex task into smaller, more manageable steps. This is achieved through techniques such as the "Chain of Thought" and "Tree of Thoughts" methods, which involve decomposing the task into smaller sub-tasks and generating multiple possibilities for each step.'

In [67]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)

{'input': 'What are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Task decomposition is a process in which an agent breaks down a complex task into smaller, more manageable steps. This is achieved through techniques such as the "Chain of Thought" and "Tree of Thoughts" methods, which involve decomposing the task into smaller sub-tasks and generating multiple possibilities for each step.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='According to the provided context, common ways of doing task decomposition include:\n\n1. Using a Large Language Model (LLM) with simple prompting, such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel\n3. W